In [ ]:
# PostGreSQL data base
# Azure
# Google
# AWS
# Dockers
# Terraform

## Assistant API- Knowledge Retrieval
* External information can be provided to LLM
    * Fine Tune: Teaching the LLM new information (very expensve & requires expert skills)
    * Knowledge Retrieval: External informaion is converted into embeddings and saving in vector data bases. And embeddings related to the questions are provided in responses.
    * Function Calling

### Providing information to LLM 
* At Assistant Level: Publically available and can be accessed by other assistants.
* At Thread Level: Available only at particulat Thread Level